In [106]:
import re
import numpy
from nltk.corpus import wordnet
import nltk
import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer

#python -m nltk.downloader all

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier

import pickle

from pprint import pprint
from time import time
import logging

size_glove = 100

# using coordinate-wise minimum - minimum from all first elements in vector and so on

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def load_stop_words(path_to_file):
    with open(path_to_file, mode="r", encoding="ISO-8859-1") as file:
        lines = file.readlines()
    return [stopword.strip() for stopword in lines]
    
def get_lemmatized_words(sentence, stopwords):
    _temp = []
    tokens = nltk.word_tokenize(sentence)
    lmtzr = WordNetLemmatizer()
    
    for word, token in nltk.pos_tag(tokens):
        if word.strip() not in stopwords:
            _temp.append(lmtzr.lemmatize(word.strip().lower(), get_wordnet_pos(token)))
    return " ".join(_temp)


-

def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = np.array(embedding).reshape(1,-1)
    print ("Done.",len(model)," words loaded!")
    return model



def sentence_2_vec(model, sentence):
    vectors = []
    for word in sentence.split(): 
        try:
            vectors.append(model[word.strip().lower()][0])
        except:
            pass
    #print(vectors)
    vectors = np.matrix(np.array(vectors).reshape(-1, size_glove))
    #print(vectors)
    maxVector = []
    minVector = []
    for i in range(size_glove):
        maxVector.append(np.max(vectors[:,i]))
        minVector.append(np.min(vectors[:,i]))
    return maxVector + minVector

def process_file(X_train, model):
    result_of_processing = []
    
    for sentence_in_X in X_train:
        vectors = sentence_2_vec(model, sentence=sentence_in_X)
        result_of_processing.append(vectors)
        
    return result_of_processing





In [96]:
model = loadGloveModel("./glove_data/glove.6B." + str(size_glove) + "d.txt")
X_train, y_train = load_file_to_process("./data/train_5500.label")

Loading Glove Model
Done. 400000  words loaded!


In [107]:
post_X_train = np.array(process_file(X_train, model))

In [109]:
post_X_traint_X_train.shape

(5452, 200)

In [102]:
len(y_train)

5452

In [3]:
x_train[:10]

['How do serfdom develop in and then leave Russia ?',
 'What film feature the character Popeye Doyle ?',
 "How can I find a list of celebrity ' real name ?",
 'What fowl grab the spotlight after the Chinese Year of the Monkey ?',
 'What be the full form of .com ?',
 'What contemptible scoundrel steal the cork from my lunch ?',
 "What team do baseball 's St. Louis Browns become ?",
 'What be the old profession ?',
 'What be liver enzyme ?',
 'Name the scar-faced bounty hunter of The Old West .']

In [29]:
size_glove = 100
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = np.array(embedding).reshape(1,-1)
    print ("Done.",len(model)," words loaded!")
    return model


In [30]:
model = loadGloveModel("./glove_data/glove.6B." + str(size_glove) + "d.txt")

Loading Glove Model
Done. 400000  words loaded!


In [88]:
def sentence_2_vec(model, sentence):
    vectors = []
    for word in sentence.split(): 
        try:
            vectors.append(model[word.strip().lower()][0])
        except:
            pass
    #print(vectors)
    vectors = np.matrix(np.array(vectors).reshape(-1, size_glove))
    #print(vectors)
    maxVector = []
    minVector = []
    for i in range(size_glove):
        maxVector.append(np.max(vectors[:,i]))
        minVector.append(np.min(vectors[:,i]))
    return maxVector + minVector

In [91]:
sentence_2_vec(model, x_train[0:2])

AttributeError: 'list' object has no attribute 'split'

In [92]:
X_train, y_train = load_file_to_process("./data/train_5500.label")


NameError: name 'load_file_to_process' is not defined

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier

from pprint import pprint
from time import time
import logging

In [12]:
pipeline1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()), # Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__stop_words': ({'english'}, None),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (2, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__loss': ('perceptron', 'hinge'),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__n_iter': (10, 25),
}

grid_search = GridSearchCV(pipeline1, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline1.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(x_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__loss': ('perceptron', 'hinge'),
 'clf__n_iter': (10, 25),
 'clf__penalty': ('l2', 'elasticnet'),
 'tfidf__norm': ('l1', 'l2'),
 'tfidf__use_idf': (True, False),
 'vect__max_features': (None, 5000, 10000, 50000),
 'vect__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'vect__stop_words': ({'english'}, None)}
Fitting 3 folds for each of 768 candidates, totalling 2304 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 2304 out of 2304 | elapsed:  6.2min finished


done in 371.436s

Best score: 0.866
Best parameters set:
	clf__loss: 'hinge'
	clf__n_iter: 25
	clf__penalty: 'elasticnet'
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_features: 10000
	vect__ngram_range: (1, 2)
	vect__stop_words: {'english'}


In [15]:
TfidfVectorizer().get_params().keys()

dict_keys(['strip_accents', 'vocabulary', 'input', 'lowercase', 'analyzer', 'max_df', 'token_pattern', 'norm', 'decode_error', 'min_df', 'binary', 'max_features', 'preprocessor', 'stop_words', 'use_idf', 'tokenizer', 'sublinear_tf', 'smooth_idf', 'dtype', 'encoding', 'ngram_range'])

In [18]:
pipeline1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier()), # Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'tfidf__stop_words': ({'english'}, None),
    'tfidf__max_features': (None, 5000, 10000, 50000),
    'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__max_df': (0.5, 1.0 , 2.0), # ignore terms that have a document frequency strictly higher than the given threshold
    'tfidf__norm': ('l1', 'l2'),
    'clf__loss': ('perceptron', 'hinge'),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__n_iter': (10, 25),
}

grid_search = GridSearchCV(pipeline1, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline1.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(x_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'clf__loss': ('perceptron', 'hinge'),
 'clf__n_iter': (10, 25),
 'clf__penalty': ('l2', 'elasticnet'),
 'tfidf__max_df': (0.5, 1.0, 2.0),
 'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__norm': ('l1', 'l2'),
 'tfidf__stop_words': ({'english'}, None),
 'tfidf__use_idf': (True, False)}
Fitting 3 folds for each of 2304 candidates, totalling 6912 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 6912 out of 6912 | elapsed: 18.8min finished


done in 1125.914s

Best score: 0.868
Best parameters set:
	clf__loss: 'hinge'
	clf__n_iter: 25
	clf__penalty: 'elasticnet'
	tfidf__max_df: 1.0
	tfidf__max_features: 10000
	tfidf__ngram_range: (1, 2)
	tfidf__norm: 'l2'
	tfidf__stop_words: {'english'}
	tfidf__use_idf: True


In [27]:
print ([key for key in TfidfVectorizer().get_params().keys() if key in CountVectorizer().get_params().keys()])

['strip_accents', 'vocabulary', 'input', 'lowercase', 'analyzer', 'max_df', 'token_pattern', 'decode_error', 'min_df', 'binary', 'max_features', 'preprocessor', 'stop_words', 'tokenizer', 'dtype', 'encoding', 'ngram_range']


In [8]:
KNeighborsClassifier(metric="mahalanobis")

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='mahalanobis',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [ ]:
classifiers = {
    # Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification)
    "MultinomialNB" :MultinomialNB(),
    "KNeighborsClassifier":KNeighborsClassifier(),
    
    # Classifier implementing a vote among neighbors within a given radius
    "RadiusNeighborsClassifier":RadiusNeighborsClassifier(),
    
    # Multi-layer Perceptron classifier.
    "MLPClassifier":MLPClassifier(activation='relu', solver='sgd', learning_rate='adaptive'),
    
    # A random forest is a meta estimator that fits a number of decision tree classifiers 
    # on various sub-samples of the dataset and use averaging to improve the predictive 
    # accuracy and control over-fitting.
    "RandomForestClassifier":RandomForestClassifier(min_samples_leaf=10),
    
    #An AdaBoost [1] classifier is a meta-estimator that begins by fitting a classifier 
    # on the original dataset and then fits additional copies of the classifier on the same dataset 
    # but where the weights of incorrectly classified instances are adjusted such that subsequent 
    # classifiers focus more on difficult cases.
    "AdaBoostClassifier":AdaBoostClassifier(),
    "SGDClassifier":SGDClassifier()
} 

vectorizers = {
    # Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification)
    "TfidfVectorizer" :TfidfVectorizer(),
    "CountVectorizer":CountVectorizer()
}

best_scores = []

for name_classifier, classifier in classifiers.items():
    for name_vectorizer, vectorizer in vectorizers.items():
        pipeline = Pipeline([
            ('tfidf', vectorizer),
            ('clf', classifier) ])

        parameters = {
            'tfidf__stop_words': ({'english'}, None),
            'tfidf__max_features': (None, 5000, 10000, 50000),
            'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),  # unigrams or bigrams
            'tfidf__lowercase': (True, False),
            'tfidf__max_df': (0.5, 1.0 , 2.0), # ignore terms that have a document frequency strictly higher than the given threshold
        }

        grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

        print("Performing grid search...")
        print("pipeline:", [name for name, _ in pipeline.steps])
        print("parameters:")
        pprint(parameters)
        t0 = time()
        grid_search.fit(x_train, y_train)
        print("done in %0.3fs" % (time() - t0))
        print()

        best_scores.append((name_classifier, name_vectorizer, grid_search.best_score_, (time() - t0), grid_search.best_estimator_.get_params()))

        print("Best score: %0.3f" % grid_search.best_score_)
        print("Best parameters set:")
        best_parameters = grid_search.best_estimator_.get_params()
        for param_name in sorted(parameters.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))
            
print (best_scores)

Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__lowercase': (True, False),
 'tfidf__max_df': (0.5, 1.0, 2.0),
 'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__stop_words': ({'english'}, None)}
Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.3min finished


done in 80.396s

Best score: 0.853
Best parameters set:
	tfidf__lowercase: False
	tfidf__max_df: 0.5
	tfidf__max_features: None
	tfidf__ngram_range: (1, 2)
	tfidf__stop_words: None
Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__lowercase': (True, False),
 'tfidf__max_df': (0.5, 1.0, 2.0),
 'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__stop_words': ({'english'}, None)}
Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.1min finished


done in 64.436s

Best score: 0.867
Best parameters set:
	tfidf__lowercase: False
	tfidf__max_df: 0.5
	tfidf__max_features: None
	tfidf__ngram_range: (1, 2)
	tfidf__stop_words: None
Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__lowercase': (True, False),
 'tfidf__max_df': (0.5, 1.0, 2.0),
 'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__stop_words': ({'english'}, None)}
Fitting 3 folds for each of 144 candidates, totalling 432 fits


...

done in 191.362s

Best score: 0.757
Best parameters set:
	tfidf__lowercase: False
	tfidf__max_df: 1.0
	tfidf__max_features: 5000
	tfidf__ngram_range: (1, 2)
	tfidf__stop_words: {'english'}
Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__lowercase': (True, False),
 'tfidf__max_df': (0.5, 1.0, 2.0),
 'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__stop_words': ({'english'}, None)}
Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  2.3min finished


done in 139.894s

Best score: 0.601
Best parameters set:
	tfidf__lowercase: True
	tfidf__max_df: 0.5
	tfidf__max_features: 5000
	tfidf__ngram_range: (1, 2)
	tfidf__stop_words: None
Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__lowercase': (True, False),
 'tfidf__max_df': (0.5, 1.0, 2.0),
 'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__stop_words': ({'english'}, None)}
Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  2.5min finished


done in 151.636s

Best score: 0.573
Best parameters set:
	tfidf__lowercase: True
	tfidf__max_df: 0.5
	tfidf__max_features: 5000
	tfidf__ngram_range: (1, 1)
	tfidf__stop_words: None
Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__lowercase': (True, False),
 'tfidf__max_df': (0.5, 1.0, 2.0),
 'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__stop_words': ({'english'}, None)}
Fitting 3 folds for each of 144 candidates, totalling 432 fits
